In [2]:
from tqdm import tqdm
import os
import numpy as np
import time
import random
import pandas as pd
from scipy import sparse
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings("ignore")

In [3]:
data_org_dir = 'data/data_tencent/'
data_prep_dir = 'data/preprocess/'
# data_prep_dir = 'data/sample/'
sample_test = False

In [4]:
if sample_test:
    df_data = pd.read_csv(data_prep_dir + 'train_feat_merge_mini.csv')
else:
    df_data = pd.read_csv(data_prep_dir + 'train_feat_merge.csv')

train_idx_lst = list(df_data[df_data['n_parts'] != 1].index)
valid_idx_lst = list(df_data[df_data['n_parts'] == 1].index)

# 单个特征

In [8]:
# construct x matrix
x_train = sparse.load_npz(data_prep_dir + 'train_x_sparse_selection.npz')
x_valid = sparse.load_npz(data_prep_dir + 'valid_x_sparse_selection.npz')
# train_x_sparse_cross_selection.npz
# valid_x_sparse_cross_selection.npz


print('x_train:', x_train.shape)
print('x_valid:', x_valid.shape)

# construct y vector
y_train = np.array(df_data[df_data['n_parts'] != 1]['label'])
y_valid = np.array(df_data[df_data['n_parts'] == 1]['label'])

print('y_train:', y_train.shape)
print('y_valid:', y_valid.shape)

x_train: (7038840, 800)
x_valid: (1759974, 800)
y_train: (7038840,)
y_valid: (1759974,)


## LogisticRegression

In [16]:
# 模型定义
clf = LogisticRegression(random_state=1, penalty='l2', C=10, solver='lbfgs',
                         tol=1e-4, max_iter=10000, verbose=2)
clf.fit(x_train, y_train)
 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 46.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 46.7min finished


LogisticRegression(C=10, max_iter=10000, random_state=1, verbose=2)

In [17]:
y_valid_pred = clf.predict(x_valid)

print('===================Test Set Performance===================')
print(pd.Series(y_valid_pred).describe())
print('Test AUC', roc_auc_score(y_valid, y_valid_pred))
print('Test Logloss', log_loss(y_valid, y_valid_pred))

===================Test Set Performance===================
count    1.759974e+06
mean     1.259110e-03
std      3.546160e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
dtype: float64
Test AUC 0.5060704749217427
Test Logloss 1.6591814964471123


In [ ]:
y_valid_pred = []

## LightGBM

In [9]:
# 模型定义
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=40, max_depth=-1, learning_rate=0.1, 
                n_estimators=10000, subsample_for_bin=200000, objective=None, 
                class_weight=None, min_split_gain=0.0, min_child_weight=0.001, 
                min_child_samples=20, subsample=0.7, subsample_freq=1, 
                colsample_bytree=0.7, 
                reg_alpha=6, reg_lambda=3,
                random_state=2018, n_jobs=-1, silent=False)

In [11]:
# 模型训练
clf.fit(x_train, y_train, eval_set = [(x_train, y_train), (x_valid, y_valid)], 
        eval_names =['train','valid'],
        eval_metric='auc', early_stopping_rounds=50)

[LightGBM] [Info] Number of positive: 337465, number of negative: 6701375
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 6.726142 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1756
[LightGBM] [Info] Number of data points in the train set: 7038840, number of used features: 800
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.047943 -> initscore=-2.988606
[LightGBM] [Info] Start training from score -2.988606
[1]	train's auc: 0.641229	train's binary_logloss: 0.189886	valid's auc: 0.640498	valid's binary_logloss: 0.190082
Training until validation scores don't improve for 50 rounds
[2]	train's auc: 0.66071	train's binary_logloss: 0.188155	valid's auc: 0.659761	valid's binary_logloss: 0.18835
[3]	train's auc: 0.66179	train's binary_logloss: 0.186925	valid's auc: 0.660796	valid's binary_logloss: 0.187111
[4]	train's auc: 0.66314	train's 

[68]	train's auc: 0.714034	train's binary_logloss: 0.175509	valid's auc: 0.712814	valid's binary_logloss: 0.175812
[69]	train's auc: 0.714337	train's binary_logloss: 0.17546	valid's auc: 0.71313	valid's binary_logloss: 0.175761
[70]	train's auc: 0.714535	train's binary_logloss: 0.175419	valid's auc: 0.713293	valid's binary_logloss: 0.175723
[71]	train's auc: 0.714837	train's binary_logloss: 0.175371	valid's auc: 0.713602	valid's binary_logloss: 0.175674
[72]	train's auc: 0.715156	train's binary_logloss: 0.175324	valid's auc: 0.713886	valid's binary_logloss: 0.175632
[73]	train's auc: 0.715352	train's binary_logloss: 0.175281	valid's auc: 0.714078	valid's binary_logloss: 0.175592
[74]	train's auc: 0.715577	train's binary_logloss: 0.175242	valid's auc: 0.714337	valid's binary_logloss: 0.17555
[75]	train's auc: 0.715742	train's binary_logloss: 0.175195	valid's auc: 0.714506	valid's binary_logloss: 0.175504
[76]	train's auc: 0.715926	train's binary_logloss: 0.175155	valid's auc: 0.714661	v

[140]	train's auc: 0.726183	train's binary_logloss: 0.173346	valid's auc: 0.723904	valid's binary_logloss: 0.173811
[141]	train's auc: 0.726313	train's binary_logloss: 0.173326	valid's auc: 0.724011	valid's binary_logloss: 0.173795
[142]	train's auc: 0.726398	train's binary_logloss: 0.173305	valid's auc: 0.724082	valid's binary_logloss: 0.173777
[143]	train's auc: 0.726488	train's binary_logloss: 0.173289	valid's auc: 0.724174	valid's binary_logloss: 0.173762
[144]	train's auc: 0.726614	train's binary_logloss: 0.173271	valid's auc: 0.724279	valid's binary_logloss: 0.173747
[145]	train's auc: 0.726688	train's binary_logloss: 0.173255	valid's auc: 0.724336	valid's binary_logloss: 0.173734
[146]	train's auc: 0.72681	train's binary_logloss: 0.173234	valid's auc: 0.724478	valid's binary_logloss: 0.173713
[147]	train's auc: 0.726885	train's binary_logloss: 0.173218	valid's auc: 0.724544	valid's binary_logloss: 0.173699
[148]	train's auc: 0.726997	train's binary_logloss: 0.173201	valid's auc:

[211]	train's auc: 0.732173	train's binary_logloss: 0.172281	valid's auc: 0.729009	valid's binary_logloss: 0.172913
[212]	train's auc: 0.73234	train's binary_logloss: 0.172257	valid's auc: 0.72915	valid's binary_logloss: 0.172892
[213]	train's auc: 0.732401	train's binary_logloss: 0.172245	valid's auc: 0.729218	valid's binary_logloss: 0.172881
[214]	train's auc: 0.732513	train's binary_logloss: 0.17223	valid's auc: 0.729312	valid's binary_logloss: 0.172869
[215]	train's auc: 0.732667	train's binary_logloss: 0.172212	valid's auc: 0.729456	valid's binary_logloss: 0.172853
[216]	train's auc: 0.732763	train's binary_logloss: 0.172197	valid's auc: 0.729528	valid's binary_logloss: 0.172841
[217]	train's auc: 0.732819	train's binary_logloss: 0.172187	valid's auc: 0.729564	valid's binary_logloss: 0.172834
[218]	train's auc: 0.732921	train's binary_logloss: 0.172172	valid's auc: 0.729643	valid's binary_logloss: 0.172821
[219]	train's auc: 0.732985	train's binary_logloss: 0.172158	valid's auc: 0

[282]	train's auc: 0.736672	train's binary_logloss: 0.171521	valid's auc: 0.732414	valid's binary_logloss: 0.172329
[283]	train's auc: 0.736717	train's binary_logloss: 0.171514	valid's auc: 0.73244	valid's binary_logloss: 0.172323
[284]	train's auc: 0.736756	train's binary_logloss: 0.171507	valid's auc: 0.732462	valid's binary_logloss: 0.17232
[285]	train's auc: 0.736878	train's binary_logloss: 0.171491	valid's auc: 0.73257	valid's binary_logloss: 0.172306
[286]	train's auc: 0.736942	train's binary_logloss: 0.171482	valid's auc: 0.732607	valid's binary_logloss: 0.172298
[287]	train's auc: 0.736989	train's binary_logloss: 0.171473	valid's auc: 0.732643	valid's binary_logloss: 0.172292
[288]	train's auc: 0.737044	train's binary_logloss: 0.171466	valid's auc: 0.732669	valid's binary_logloss: 0.172288
[289]	train's auc: 0.737085	train's binary_logloss: 0.171459	valid's auc: 0.732712	valid's binary_logloss: 0.172282
[290]	train's auc: 0.737121	train's binary_logloss: 0.171452	valid's auc: 0

...

[1915]	train's auc: 0.772879	train's binary_logloss: 0.165249	valid's auc: 0.743736	valid's binary_logloss: 0.170219
[1916]	train's auc: 0.772896	train's binary_logloss: 0.165246	valid's auc: 0.743737	valid's binary_logloss: 0.170218
[1917]	train's auc: 0.772909	train's binary_logloss: 0.165244	valid's auc: 0.743733	valid's binary_logloss: 0.170219
[1918]	train's auc: 0.772923	train's binary_logloss: 0.16524	valid's auc: 0.743735	valid's binary_logloss: 0.170218
[1919]	train's auc: 0.77295	train's binary_logloss: 0.165237	valid's auc: 0.743741	valid's binary_logloss: 0.170217
[1920]	train's auc: 0.772966	train's binary_logloss: 0.165235	valid's auc: 0.743741	valid's binary_logloss: 0.170217
[1921]	train's auc: 0.772978	train's binary_logloss: 0.165232	valid's auc: 0.743741	valid's binary_logloss: 0.170217
[1922]	train's auc: 0.772987	train's binary_logloss: 0.16523	valid's auc: 0.743738	valid's binary_logloss: 0.170217
[1923]	train's auc: 0.773002	train's binary_logloss: 0.165227	valid

[1986]	train's auc: 0.773895	train's binary_logloss: 0.165058	valid's auc: 0.743801	valid's binary_logloss: 0.170204
[1987]	train's auc: 0.773905	train's binary_logloss: 0.165055	valid's auc: 0.743801	valid's binary_logloss: 0.170203
[1988]	train's auc: 0.773915	train's binary_logloss: 0.165053	valid's auc: 0.743799	valid's binary_logloss: 0.170203
[1989]	train's auc: 0.773939	train's binary_logloss: 0.16505	valid's auc: 0.74379	valid's binary_logloss: 0.170204
[1990]	train's auc: 0.773945	train's binary_logloss: 0.165047	valid's auc: 0.743791	valid's binary_logloss: 0.170204
[1991]	train's auc: 0.773966	train's binary_logloss: 0.165043	valid's auc: 0.743794	valid's binary_logloss: 0.170203
[1992]	train's auc: 0.773983	train's binary_logloss: 0.16504	valid's auc: 0.743795	valid's binary_logloss: 0.170203
[1993]	train's auc: 0.773999	train's binary_logloss: 0.165038	valid's auc: 0.743798	valid's binary_logloss: 0.170203
[1994]	train's auc: 0.774012	train's binary_logloss: 0.165035	valid

[2057]	train's auc: 0.77491	train's binary_logloss: 0.164867	valid's auc: 0.743801	valid's binary_logloss: 0.170196
[2058]	train's auc: 0.774924	train's binary_logloss: 0.164864	valid's auc: 0.743797	valid's binary_logloss: 0.170197
[2059]	train's auc: 0.774933	train's binary_logloss: 0.164862	valid's auc: 0.7438	valid's binary_logloss: 0.170196
[2060]	train's auc: 0.77495	train's binary_logloss: 0.164859	valid's auc: 0.743797	valid's binary_logloss: 0.170197
[2061]	train's auc: 0.774967	train's binary_logloss: 0.164857	valid's auc: 0.743799	valid's binary_logloss: 0.170197
[2062]	train's auc: 0.774984	train's binary_logloss: 0.164854	valid's auc: 0.743803	valid's binary_logloss: 0.170197
[2063]	train's auc: 0.774995	train's binary_logloss: 0.164852	valid's auc: 0.743803	valid's binary_logloss: 0.170197
[2064]	train's auc: 0.775002	train's binary_logloss: 0.164849	valid's auc: 0.743805	valid's binary_logloss: 0.170196
[2065]	train's auc: 0.775021	train's binary_logloss: 0.164847	valid'

[2128]	train's auc: 0.775937	train's binary_logloss: 0.164684	valid's auc: 0.743805	valid's binary_logloss: 0.170191
[2129]	train's auc: 0.77595	train's binary_logloss: 0.164681	valid's auc: 0.743805	valid's binary_logloss: 0.17019
[2130]	train's auc: 0.775965	train's binary_logloss: 0.164678	valid's auc: 0.743807	valid's binary_logloss: 0.17019
[2131]	train's auc: 0.775984	train's binary_logloss: 0.164675	valid's auc: 0.743808	valid's binary_logloss: 0.17019
[2132]	train's auc: 0.776001	train's binary_logloss: 0.164673	valid's auc: 0.743807	valid's binary_logloss: 0.17019
[2133]	train's auc: 0.77602	train's binary_logloss: 0.16467	valid's auc: 0.743807	valid's binary_logloss: 0.170189
[2134]	train's auc: 0.776035	train's binary_logloss: 0.164667	valid's auc: 0.743801	valid's binary_logloss: 0.17019
[2135]	train's auc: 0.77605	train's binary_logloss: 0.164665	valid's auc: 0.743804	valid's binary_logloss: 0.17019
[2136]	train's auc: 0.776073	train's binary_logloss: 0.164661	valid's auc:

LGBMClassifier(colsample_bytree=0.7, n_estimators=10000, num_leaves=40,
               random_state=2018, reg_alpha=6, reg_lambda=3, silent=False,
               subsample=0.7, subsample_freq=1)

In [14]:
print('===================Test Set Performance===================')
y_valid_pred = clf.predict_proba(x_valid, num_iteration = clf.best_iteration_)[:,1]
print(pd.Series(y_valid_pred).describe())
print('Test AUC', roc_auc_score(y_valid, y_valid_pred))
print('Test Logloss', log_loss(y_valid, y_valid_pred))

===================Test Set Performance===================
count    1.759974e+06
mean     4.796217e-02
std      5.839436e-02
min      2.186377e-04
25%      2.071137e-02
50%      3.481173e-02
75%      5.510371e-02
max      9.953930e-01
dtype: float64
Test AUC 0.7438440879170812
Test Logloss 0.17018985844814913


# 加入交叉特征

In [5]:
# construct x matrix
x_train = sparse.load_npz(data_prep_dir + 'train_x_sparse_selection.npz')
x_valid = sparse.load_npz(data_prep_dir + 'valid_x_sparse_selection.npz')
# train_x_sparse_cross_selection.npz
# valid_x_sparse_cross_selection.npz
x_train_cross = sparse.load_npz(data_prep_dir + 'train_x_sparse_cross_selection.npz')
x_valid_cross = sparse.load_npz(data_prep_dir + 'valid_x_sparse_cross_selection.npz')

x_train = sparse.hstack((x_train, x_train_cross)).tocsc()
x_valid = sparse.hstack((x_valid, x_valid_cross)).tocsc()

print('x_train:', x_train.shape)
print('x_valid:', x_valid.shape)

x_train: (7038840, 9300)
x_valid: (1759974, 9300)


In [6]:
del x_train_cross, x_valid_cross

In [7]:
# construct y vector
y_train = np.array(df_data[df_data['n_parts'] != 1]['label'])
y_valid = np.array(df_data[df_data['n_parts'] == 1]['label'])

print('y_train:', y_train.shape)
print('y_valid:', y_valid.shape)

y_train: (7038840,)
y_valid: (1759974,)


## LightGBM

In [24]:
# 模型定义
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=40, max_depth=-1, learning_rate=0.1, 
                n_estimators=10000, subsample_for_bin=200000, objective=None, 
                class_weight=None, min_split_gain=0.0, min_child_weight=0.001, 
                min_child_samples=20, subsample=0.7, subsample_freq=1, 
                colsample_bytree=0.7, 
                reg_alpha=6, reg_lambda=3,
                random_state=2018, n_jobs=-1, silent=False)

In [25]:
# 模型训练
clf.fit(x_train, y_train, eval_set = [(x_train, y_train), (x_valid, y_valid)], 
        eval_names =['train','valid'],
        eval_metric='auc', early_stopping_rounds=50)

[LightGBM] [Info] Number of positive: 337465, number of negative: 6701375
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 114.529342 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 18638
[LightGBM] [Info] Number of data points in the train set: 7038840, number of used features: 9241
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.047943 -> initscore=-2.988606
[LightGBM] [Info] Start training from score -2.988606
[1]	train's auc: 0.643552	train's binary_logloss: 0.189725	valid's auc: 0.642099	valid's binary_logloss: 0.189903
Training until validation scores don't improve for 50 rounds
[2]	train's auc: 0.654497	train's binary_logloss: 0.188154	valid's auc: 0.653247	valid's binary_logloss: 0.188321
[3]	train's auc: 0.663561	train's binary_logloss: 0.186991	valid's auc: 0.661819	valid's binary_logloss: 0.187186
[4]	train's auc: 0.668731	

[68]	train's auc: 0.717159	train's binary_logloss: 0.175005	valid's auc: 0.715649	valid's binary_logloss: 0.175351
[69]	train's auc: 0.717514	train's binary_logloss: 0.174953	valid's auc: 0.716007	valid's binary_logloss: 0.1753
[70]	train's auc: 0.717738	train's binary_logloss: 0.17491	valid's auc: 0.716222	valid's binary_logloss: 0.175259
[71]	train's auc: 0.718187	train's binary_logloss: 0.174854	valid's auc: 0.716647	valid's binary_logloss: 0.175206
[72]	train's auc: 0.718467	train's binary_logloss: 0.174807	valid's auc: 0.716886	valid's binary_logloss: 0.175164
[73]	train's auc: 0.718832	train's binary_logloss: 0.174756	valid's auc: 0.717238	valid's binary_logloss: 0.175115
[74]	train's auc: 0.719127	train's binary_logloss: 0.17471	valid's auc: 0.717488	valid's binary_logloss: 0.175075
[75]	train's auc: 0.719357	train's binary_logloss: 0.174671	valid's auc: 0.717691	valid's binary_logloss: 0.175038
[76]	train's auc: 0.719607	train's binary_logloss: 0.174627	valid's auc: 0.717915	va

[140]	train's auc: 0.730597	train's binary_logloss: 0.172718	valid's auc: 0.728065	valid's binary_logloss: 0.17324
[141]	train's auc: 0.730717	train's binary_logloss: 0.172694	valid's auc: 0.728167	valid's binary_logloss: 0.17322
[142]	train's auc: 0.730809	train's binary_logloss: 0.172671	valid's auc: 0.728237	valid's binary_logloss: 0.1732
[143]	train's auc: 0.730935	train's binary_logloss: 0.172653	valid's auc: 0.728359	valid's binary_logloss: 0.173183
[144]	train's auc: 0.731031	train's binary_logloss: 0.172634	valid's auc: 0.72845	valid's binary_logloss: 0.173164
[145]	train's auc: 0.73114	train's binary_logloss: 0.172613	valid's auc: 0.728547	valid's binary_logloss: 0.173145
[146]	train's auc: 0.731256	train's binary_logloss: 0.172594	valid's auc: 0.728627	valid's binary_logloss: 0.173129
[147]	train's auc: 0.731362	train's binary_logloss: 0.172574	valid's auc: 0.728711	valid's binary_logloss: 0.173113
[148]	train's auc: 0.731461	train's binary_logloss: 0.172557	valid's auc: 0.72

[211]	train's auc: 0.736957	train's binary_logloss: 0.171584	valid's auc: 0.73332	valid's binary_logloss: 0.172291
[212]	train's auc: 0.737026	train's binary_logloss: 0.17157	valid's auc: 0.733371	valid's binary_logloss: 0.172281
[213]	train's auc: 0.737092	train's binary_logloss: 0.171558	valid's auc: 0.733434	valid's binary_logloss: 0.17227
[214]	train's auc: 0.737169	train's binary_logloss: 0.171545	valid's auc: 0.733503	valid's binary_logloss: 0.172258
[215]	train's auc: 0.737236	train's binary_logloss: 0.171534	valid's auc: 0.733559	valid's binary_logloss: 0.17225
[216]	train's auc: 0.737287	train's binary_logloss: 0.171524	valid's auc: 0.733608	valid's binary_logloss: 0.172241
[217]	train's auc: 0.737344	train's binary_logloss: 0.171515	valid's auc: 0.733643	valid's binary_logloss: 0.172234
[218]	train's auc: 0.737419	train's binary_logloss: 0.171501	valid's auc: 0.733686	valid's binary_logloss: 0.172225
[219]	train's auc: 0.737511	train's binary_logloss: 0.171485	valid's auc: 0.

[282]	train's auc: 0.741097	train's binary_logloss: 0.170849	valid's auc: 0.736238	valid's binary_logloss: 0.171761
[283]	train's auc: 0.741159	train's binary_logloss: 0.17084	valid's auc: 0.736284	valid's binary_logloss: 0.171754
[284]	train's auc: 0.741206	train's binary_logloss: 0.170833	valid's auc: 0.736309	valid's binary_logloss: 0.171749
[285]	train's auc: 0.741284	train's binary_logloss: 0.170821	valid's auc: 0.736358	valid's binary_logloss: 0.171741
[286]	train's auc: 0.741332	train's binary_logloss: 0.170812	valid's auc: 0.736386	valid's binary_logloss: 0.171735
[287]	train's auc: 0.741382	train's binary_logloss: 0.170804	valid's auc: 0.736414	valid's binary_logloss: 0.17173
[288]	train's auc: 0.741439	train's binary_logloss: 0.170795	valid's auc: 0.736449	valid's binary_logloss: 0.171726
[289]	train's auc: 0.741476	train's binary_logloss: 0.170788	valid's auc: 0.736475	valid's binary_logloss: 0.171721
[290]	train's auc: 0.741517	train's binary_logloss: 0.170779	valid's auc: 

...

[1134]	train's auc: 0.763391	train's binary_logloss: 0.166977	valid's auc: 0.743827	valid's binary_logloss: 0.170315
[1135]	train's auc: 0.76341	train's binary_logloss: 0.166973	valid's auc: 0.74383	valid's binary_logloss: 0.170314
[1136]	train's auc: 0.763427	train's binary_logloss: 0.16697	valid's auc: 0.743833	valid's binary_logloss: 0.170314
[1137]	train's auc: 0.763446	train's binary_logloss: 0.166967	valid's auc: 0.743829	valid's binary_logloss: 0.170314
[1138]	train's auc: 0.763461	train's binary_logloss: 0.166962	valid's auc: 0.743836	valid's binary_logloss: 0.170312
[1139]	train's auc: 0.763476	train's binary_logloss: 0.166959	valid's auc: 0.743841	valid's binary_logloss: 0.170311
[1140]	train's auc: 0.763495	train's binary_logloss: 0.166956	valid's auc: 0.743849	valid's binary_logloss: 0.170309
[1141]	train's auc: 0.763517	train's binary_logloss: 0.166952	valid's auc: 0.743855	valid's binary_logloss: 0.170308
[1142]	train's auc: 0.763537	train's binary_logloss: 0.166949	valid

[1205]	train's auc: 0.764697	train's binary_logloss: 0.16674	valid's auc: 0.744046	valid's binary_logloss: 0.170269
[1206]	train's auc: 0.764722	train's binary_logloss: 0.166737	valid's auc: 0.744043	valid's binary_logloss: 0.170268
[1207]	train's auc: 0.764737	train's binary_logloss: 0.166734	valid's auc: 0.744039	valid's binary_logloss: 0.170269
[1208]	train's auc: 0.764753	train's binary_logloss: 0.16673	valid's auc: 0.744042	valid's binary_logloss: 0.170268
[1209]	train's auc: 0.764768	train's binary_logloss: 0.166727	valid's auc: 0.744034	valid's binary_logloss: 0.170269
[1210]	train's auc: 0.764793	train's binary_logloss: 0.166723	valid's auc: 0.744037	valid's binary_logloss: 0.170269
[1211]	train's auc: 0.764811	train's binary_logloss: 0.16672	valid's auc: 0.744042	valid's binary_logloss: 0.170269
[1212]	train's auc: 0.76483	train's binary_logloss: 0.166717	valid's auc: 0.744047	valid's binary_logloss: 0.170267
[1213]	train's auc: 0.764842	train's binary_logloss: 0.166715	valid'

[1276]	train's auc: 0.765983	train's binary_logloss: 0.166516	valid's auc: 0.744174	valid's binary_logloss: 0.17024
[1277]	train's auc: 0.766	train's binary_logloss: 0.166512	valid's auc: 0.744172	valid's binary_logloss: 0.170239
[1278]	train's auc: 0.766013	train's binary_logloss: 0.16651	valid's auc: 0.744177	valid's binary_logloss: 0.170239
[1279]	train's auc: 0.766036	train's binary_logloss: 0.166506	valid's auc: 0.744175	valid's binary_logloss: 0.170239
[1280]	train's auc: 0.766051	train's binary_logloss: 0.166502	valid's auc: 0.744187	valid's binary_logloss: 0.170235
[1281]	train's auc: 0.766065	train's binary_logloss: 0.166499	valid's auc: 0.744184	valid's binary_logloss: 0.170235
[1282]	train's auc: 0.76608	train's binary_logloss: 0.166496	valid's auc: 0.74418	valid's binary_logloss: 0.170235
[1283]	train's auc: 0.766093	train's binary_logloss: 0.166493	valid's auc: 0.744174	valid's binary_logloss: 0.170236
[1284]	train's auc: 0.766113	train's binary_logloss: 0.16649	valid's au

[1347]	train's auc: 0.767213	train's binary_logloss: 0.166295	valid's auc: 0.744245	valid's binary_logloss: 0.170217
[1348]	train's auc: 0.767239	train's binary_logloss: 0.166291	valid's auc: 0.744241	valid's binary_logloss: 0.170217
[1349]	train's auc: 0.767248	train's binary_logloss: 0.166288	valid's auc: 0.744235	valid's binary_logloss: 0.170218
[1350]	train's auc: 0.767272	train's binary_logloss: 0.166284	valid's auc: 0.744243	valid's binary_logloss: 0.170217
[1351]	train's auc: 0.76729	train's binary_logloss: 0.166282	valid's auc: 0.744246	valid's binary_logloss: 0.170217
[1352]	train's auc: 0.767305	train's binary_logloss: 0.166279	valid's auc: 0.744247	valid's binary_logloss: 0.170217
[1353]	train's auc: 0.767322	train's binary_logloss: 0.166276	valid's auc: 0.744252	valid's binary_logloss: 0.170217
[1354]	train's auc: 0.767337	train's binary_logloss: 0.166274	valid's auc: 0.744249	valid's binary_logloss: 0.170217
[1355]	train's auc: 0.767358	train's binary_logloss: 0.166271	val

LGBMClassifier(colsample_bytree=0.7, n_estimators=10000, num_leaves=40,
               random_state=2018, reg_alpha=6, reg_lambda=3, silent=False,
               subsample=0.7, subsample_freq=1)

In [26]:
print('===================Test Set Performance===================')
y_valid_pred = clf.predict_proba(x_valid, num_iteration = clf.best_iteration_)[:,1]

print(pd.Series(y_valid_pred).describe())
print('Test AUC', roc_auc_score(y_valid, y_valid_pred))
print('Test Logloss', log_loss(y_valid, y_valid_pred))

===================Test Set Performance===================
count    1.759974e+06
mean     4.798085e-02
std      5.781655e-02
min      2.774193e-04
25%      2.066312e-02
50%      3.498627e-02
75%      5.554607e-02
max      9.938129e-01
dtype: float64
Test AUC 0.7442824433402417
Test Logloss 0.17021078701055584


## LogisticRegression

In [8]:
# 模型定义与训练
clf = LogisticRegression(random_state=1, penalty='l2', C=10, solver='lbfgs',
                         tol=1e-4, max_iter=10000, verbose=2)
clf.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 365.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 365.9min finished


LogisticRegression(C=10, max_iter=10000, random_state=1, verbose=2)

In [10]:
y_valid_pred = clf.predict(x_valid)

print('===================Test Set Performance===================')
print(pd.Series(y_valid_pred).describe())
print('Test AUC', roc_auc_score(y_valid, y_valid_pred))
print('Test Logloss', log_loss(y_valid, y_valid_pred))

===================Test Set Performance===================
count    1.759974e+06
mean     1.844345e-03
std      4.290623e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
dtype: float64
Test AUC 0.5106611993494088
Test Logloss 1.6484665776988547
